In [5]:
%pip install dwave-system==1.18.0
%pip install dwave-system[drivers] --extra-index-url https://pypi.dwavesys.com/simple
%pip install dimod==0.11.0
%pip install dwave_qbsolv

%pip install dwave-ocean-sdk>=6.3.0
%pip install mip==1.13.0
%pip install plotly==5.6.0
%pip install streamlit==1.9.2
%pip install tabulate==0.8.9

Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.dwavesys.com/simple
Note: you may need to restart the kernel to use updated packages.
  Using cached dimod-0.11.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.7 MB)
  Attempting uninstall: dimod
    Found existing installation: dimod 0.12.3
    Uninstalling dimod-0.12.3:
      Successfully uninstalled dimod-0.12.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dwave-system 1.18.0 requires dimod<0.14.0,>=0.12.0, but you have dimod 0.11.0 which is incompatible.
dwave-samplers 1.0.0 requires dimod<0.13.0,>=0.12.0, but you have dimod 0.11.0 which is incompatible.
dwave-qbsolv 0.3.4 requires dimod<0.11.0,>=0.8.1, but you have dimod 0.11.0 which is incompatible.
dwave-preprocessing 0.5.4 requires dimod<0.13.0,>=0.12.2, but you

In [6]:
import os
import time
import random


import networkx as nx
import matplotlib.pyplot as plt

from dwave_qbsolv import QBSolv
from dwave.system import DWaveSampler
import dimod
import hybrid

import MapGraph


In [69]:
def create_street_network(row, column, show=False): 
    G = MapGraph.make_graph(row, column)
    MapGraph.add_traffic_weight(G)
    MapGraph.add_time_travel(G)

    if show:
        pos = nx.spring_layout(G)
        nx.draw_networkx(G, pos, with_labels=True)
        plt.show()
    
    return G


def create_dummy_origins_and_dest(n_cars, row, column):
    """ Assign random origin and destination to each car. """
    origin_cars = [random.randint(0, row*column-1) for x in range(n_cars)]
    dest_cars = [random.randint(0, row*column-1) for x in range(n_cars)]
    
    return origin_cars, dest_cars


def create_routes(n_cars, n_routes, g, origin_cars, dest_cars):
    # Three (diverse?) routes for each car
    routes = []
    streets = []
    for n_c in range(n_cars):        
        paths = MapGraph.find_tre_shortest_paths(g, 
                                                origin_cars[n_c], 
                                                dest_cars[n_c], 
                                                overlap=5, 
                                                weight="time travel")
        path_edges = MapGraph.paths_edges(paths)
       
        
        for j in range(n_routes):
            routes.append(paths[j])  # Nodes
            streets.append(path_edges[j])  # Edges
            
    return routes, streets


def create_cars_variables(n_cars: int, routes: list, n_routes: int):
    """
        Return:
          cars_vars: ['q00', 'q01', 'q02', 'q10', ...]
          cars_routes:  {'q00': [6, 7, 12, 11], 'q01': [6, 11], ...
    """
    cars_variables = []
    cars_routes = {}
    
    running_var = 0
    for i in range(n_cars):
        for j in range(n_routes):
            running_var = i * 3 + j
            cars_variables.append(f'q{i}{j}')  # car, route
            cars_routes[cars_variables[running_var]] = routes[running_var]
                        
    return cars_variables, cars_routes

In [70]:
# Get D-Wave token
TOKEN = os.environ['DWAVE_APITOKEN']

# Problem specifications
N_CARS = 5
N_ROUTES = 3

# Street network
G_ROW = 3
G_COLUMN = 5 
G = create_street_network(G_ROW, G_COLUMN)  

ORIGIN_CARS, DEST_CARS = create_dummy_origins_and_dest(N_CARS, G_ROW, G_COLUMN)

# Create routes and assign corresponding streets
ROUTES, STREETS = create_routes(N_CARS, N_ROUTES, G, ORIGIN_CARS, DEST_CARS)

assert len(ROUTES) == (N_CARS * N_ROUTES)

# Prepare key variables
cars_vars, cars_routes = create_cars_variables(N_CARS, ROUTES, N_ROUTES)

**What does QUBO look like?**

Objective: Minimize the traffic flow $\Leftrightarrow$ Minimize number of overlapping
segments between assigned routes for each car.

Note: Here the segments are the streets (edges in the graph).

In [60]:
# classmethod BinaryQuadraticModel.from_qubo(Q: Mapping, offset: float = 0)
#     Q – Coefficients of a quadratic unconstrained binary optimization (QUBO) problem 
#         as a dict of form {(u, v): bias, ...}, where u, v, are binary-valued variables 
#         and bias is their associated coefficient.
#
#     offset (optional, default=0.0) – Constant offset applied to the model.

# QUBO
#   q_ij: car i takes route (= a list of segments) j

# def create_qubo():

qubo = {}




In [61]:
cars_vars

['q00',
 'q01',
 'q02',
 'q10',
 'q11',
 'q12',
 'q20',
 'q21',
 'q22',
 'q30',
 'q31',
 'q32',
 'q40',
 'q41',
 'q42']